# Parsing APIs Example

## Intro

Now we will take a look on a real data. When you parse data from web you will often meet API based web-pages. 

For example [zalando.fr](https://www.zalando.fr/accueil-homme/) is API based web-page. 

In this guided lab you will learn how to obtain the links from webpages and extract the data. Read through this doc, execute the cells in order and make sure you understand the explanations. 

*Note: This guided lab uses Google Chrome. Other browsers like Safari and Firefox have similar tools for developers but they work differently. To save your time in following this lab, it is strongly recommended that you install and use Google Chrome.*

## Obtaining the link

Zalando is discount e-store where you can buy clothes and accesories with discount. When we go to the web-page, we can choose different sections. First the general process will be shown using [Children section](https://www.zalando.fr/accueil-enfant/) as example.

Here we will parse data about promotions only. Therefore, final output will be the DataFrame with all the goods under discount.

[![Image from Gyazo](https://i.gyazo.com/fa4874d8e81c7570273bbfb853d66308.png)](https://gyazo.com/fa4874d8e81c7570273bbfb853d66308)


We go to Promos page. Right click of mouse shows us a list of actions possible, from which we select Inspect.

<img src='https://i.gyazo.com/bccbd11d69c9040dc98758d443e32052.png' width="400">


You will see the menu dropdown on the right side or on the bottom of the window. There you should click on Network:


[![Image from Gyazo](https://i.gyazo.com/f7e0db81cbfee67694183d1a7640bf81.png)](https://gyazo.com/f7e0db81cbfee67694183d1a7640bf81)

Right after the developer part will change showing the files behind the page. In order to obtain only useful files we select the following settings:
1. Preserve Log
2. Select XHR files.

[![Image from Gyazo](https://i.gyazo.com/9a899d4441d9d93e795f79747f1e47d5.png)](https://gyazo.com/9a899d4441d9d93e795f79747f1e47d5)

In order to obtain some files we need to scrool down and go forward to second page. 

[![Image from Gyazo](https://i.gyazo.com/0956eb3d5125075a236c9a439c7749c7.png)](https://gyazo.com/0956eb3d5125075a236c9a439c7749c7)

In the Network panel you can see the following files being uploaded. All the data on the web-page is uploaded from the json file, which is one of the following. It is important to understand which file contains what kind of information. 

<a href="https://gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c"><img src="https://i.gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c.png" alt="Image from Gyazo" width="724.8"/></a>

When you find what kind of information you need for the data to be uploaded you just test it. Here we need the article... file:

<a href="https://gyazo.com/78b35bf492994b3f35c0564a21da202a"><img src="https://i.gyazo.com/78b35bf492994b3f35c0564a21da202a.png" alt="Image from Gyazo" width="727.2"/></a>

When we test the link in Chrome inkognito mode we obtain the proper json file:


<a href="https://gyazo.com/b60453fa98454fa29771c731a5174443"><img src="https://i.gyazo.com/b60453fa98454fa29771c731a5174443.png" alt="Image from Gyazo" width="1530.4"/></a>

In order to change the objects in the json file (kind of pagination), you need to change the offset (the number of the first element on the page). in fact, if you take a look on the link, it is easy to unerstand the structure of the link.

# Reading the data

Now the party rocks! When we know how can we obtain the data, it is not a problem to obtain the whole database with all the data from the web-page.
In this lab you will collect your database of Zalando products. You select which goods you want to track. You can define as many filters to your data as you want. Just make sure that the data represents the filters.




In [190]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize



In [191]:
# Paste the url you obtained for your data
url='https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset=84&sort=sale'


#### Collect first 84 object of the of the data (1st page)

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having *family_articles, flags, media* and *sizes* remaining lists (they are exceptions). Hint: use the headers parameter to get the data!

In [192]:
# headers definition
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'

In [193]:
# Your code
import urllib.request
from urllib.parse import urlencode, quote_plus

def get_zalando_page_info(offset=84, sort='sale'):
    
    url = 'https://www.zalando.fr/api/catalog/articles/?'
    params = urlencode({'categories':'promo-enfant', 'limit': 84, 'offset': offset, 'sort': sort}, quote_via=quote_plus)
    
    print(url + params)
    
    req = urllib.request.Request(
        url + params, 
        data=None, 
        headers={
            'User-Agent': user_agent
        }
    )

    f = urllib.request.urlopen(req)
    return json.loads(f.read().decode('utf-8'))


get_zalando_page_info()

https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=84&sort=popularity


{'total_count': 26738,
 'pagination': {'page_count': 319, 'current_page': 2, 'per_page': 84},
 'sort': 'popularity',
 'articles': [{'sku': 'LE226G00Z-A11',
   'name': 'T-shirt imprimé - white',
   'price': {'original': '22,95\xa0€',
    'promotional': '14,95\xa0€',
    'has_different_prices': True,
    'has_different_original_prices': False,
    'has_different_promotional_prices': True,
    'has_discount_on_selected_sizes_only': False},
   'sizes': ['3a', '4a', '5a', '6a', '8a', '10a', '12a', '14a', '16a'],
   'url_key': 'levis-t-shirt-imprime-white-le226g00z-a11',
   'media': [{'path': 'LE/22/6G/00/ZA/11/LE226G00Z-A11@5.jpg',
     'role': 'DEFAULT',
     'packet_shot': True}],
   'brand_name': "Levi's®",
   'is_premium': False,
   'family_articles': [],
   'flags': [{'key': 'campaign',
     'value': '-20% EXTRA',
     'tracking_value': 'fr_ss20_eoss_2020_27'},
    {'key': 'discountRate',
     'value': '-35%',
     'tracking_value': 'discount rate'}],
   'product_group': 'clothing',
  

#### Collect all the objects from selected filters. Total number of pages can be found in the same json. Use *sku* column as index.

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having family_articles, flags, media and sizes remaining lists (they are exceptions).

In [194]:
# Get the total number of pages

response = get_zalando_page_info()
total_pages = response['pagination']['page_count']

total_pages
# Your code

frames = []


for page in range(2, total_pages+1):
    
    response = get_zalando_page_info(page*84)
    articles = pd.json_normalize(response['articles'])
    frames.append(articles)

data_set = pd.concat(frames)

data_set.set_index('sku', inplace=True)
    
data_set

https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=84&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=168&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=252&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=336&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=420&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=504&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=588&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=672&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=756&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=840&sort=sale
https://www

https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=7056&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=7140&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=7224&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=7308&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=7392&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=7476&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=7560&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=7644&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=7728&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=7812&sort=sale


https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=13944&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=14028&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=14112&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=14196&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=14280&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=14364&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=14448&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=14532&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=14616&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=14700&

https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=20832&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=20916&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=21000&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=21084&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=21168&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=21252&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=21336&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=21420&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=21504&sort=sale
https://www.zalando.fr/api/catalog/articles/?categories=promo-enfant&limit=84&offset=21588&

,name,sizes,url_key,media,brand_name,is_premium,family_articles,flags,product_group,delivery_promises,price.original,price.promotional,price.has_different_prices,price.has_different_original_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,amount,condition,condition_key
sku,,,,,,,,,,,,,,,,,,,
NI153I00F-J11,MINI AIR - Sac à dos - pink foam/fire pink,[One Size],nike-sportswear-mini-air-sac-a-dos-pink-foamfi...,[{'path': 'NI/15/3I/00/FJ/11/NI153I00F-J11@7.j...,Nike Sportswear,False,[],"[{'key': 'campaign', 'value': '-20% EXTRA', 't...",accessoires,[],"25,95 €","19,55 €",False,False,False,False,NaN,NaN,NaN
DX223L006-A11,ALIDA - Veste en jean - white,"[8a, 10a, 12a, 14a, 16a]",d-xel-alida-veste-en-jean-white-dx223l006-a11,[{'path': 'DX/22/3L/00/6A/11/DX223L006-A11@8.j...,D-XEL,False,[],"[{'key': 'campaign', 'value': '-20% EXTRA', 't...",clothing,[],"49,95 €","14,95 €",False,False,False,False,NaN,NaN,NaN
F0113G016-A11,LORE STRAPS MEDIUM FIT - Sandales - white,"[26, 28, 31, 33, 34, 35]",froddo-lore-straps-medium-fit-sandales-f0113g0...,[{'path': 'F0/11/3G/01/6A/11/F0113G016-A11@8.j...,Froddo,False,[],"[{'key': 'campaign', 'value': '-20% EXTRA', 't...",shoe,[],"59,95 €","29,95 €",True,False,True,False,NaN,NaN,NaN
NI126D007-Q11,SUIT CORE - veste en sweat zippée - black/white,"[6-7a, 8-9a, 10-11a, 12-13a]",nike-sportswear-suit-core-set-pantalon-de-surv...,[{'path': 'NI/12/6D/00/7Q/11/NI126D007-Q11@12....,Nike Sportswear,False,[],"[{'key': 'campaign', 'value': '-20% EXTRA', 't...",clothing,[],"54,95 €","49,45 €",False,False,False,False,NaN,NaN,NaN
L0643E00B-Q11,TENNIS PANT - Pantalon de survêtement - black/...,"[4a, 6a, 8a, 10a, 12a, 16a]",lacoste-sport-tennis-pant-pantalon-de-survetem...,[{'path': 'L0/64/3E/00/BQ/11/L0643E00B-Q11@7.j...,Lacoste Sport,False,[],"[{'key': 'campaign', 'value': '-20% EXTRA', 't...",clothing,[],"68,00 €","47,00 €",True,False,True,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EW386D03A-A11,Collants - latte,[3m],ewers-collants-latte-ew386d03a-a11,[{'path': 'EW/38/6D/03/AA/11/EW386D03A-A11@12....,Ewers,False,[],"[{'key': 'campaign', 'value': 'Prix Mini', 'tr...",underwear,[],"20,95 €","20,95 €",False,False,False,False,NaN,NaN,NaN
EW384D00W-M11,BABYTIGHT DINO 2 PACK - Collants - oliv/navy,[12m],ewers-babytight-dino-2-pack-collants-olivnavy-...,[{'path': 'EW/38/4D/00/WM/11/EW384D00W-M11@8.j...,Ewers,False,[],"[{'key': 'campaign', 'value': 'Prix Mini', 'tr...",underwear,[],"23,95 €","23,95 €",False,False,False,False,NaN,NaN,NaN
EW386D02P-T11,BABY CLOUDS 2 PACK - Collants - schwarz/grau,"[6m, 9m]",ewers-baby-clouds-2-pack-collants-ew386d02p-t11,[{'path': 'EW/38/6D/02/PT/11/EW386D02P-T11@7.j...,Ewers,False,[],"[{'key': 'campaign', 'value': 'Prix Mini', 'tr...",underwear,[],"20,95 €","20,95 €",False,False,False,False,NaN,NaN,NaN


#### Display the trending brand in DataFrame

In [219]:
# your code
data_set.groupby('brand_name').count().nlargest(1,'name')['name']



brand_name
Name it    845
Name: name, dtype: int64

#### Display the brand with maximal total discount (sum of discounts on all goods)

In [220]:
#Our data is still text. Convert prices into numbers.
# your code
data_set['price.promotional'] = data_set['price.promotional'].str.replace('\xa0€', '').str.replace(',', '.')
data_set['price.promotional'] = data_set['price.promotional'].astype(float)

data_set['price.original'] = data_set['price.original'].str.replace('\xa0€', '').str.replace(',', '.')
data_set['price.original'] = data_set['price.original'].astype(float)





AttributeError: Can only use .str accessor with string values!

In [223]:
data_set['total_discount'] = data_set['price.original'] - data_set['price.promotional']

In [254]:
data_set.groupby('brand_name').sum().nlargest(1, 'total_discount')['total_discount']


brand_name
Scotch & Soda    15294.9
Name: total_discount, dtype: float64

#### Display the brands without discount at all

In [267]:
# your code
    
brand_names = data_set.groupby('brand_name').sum()

brand_names[brand_names['total_discount']==0]

,is_premium,price.original,price.promotional,price.has_different_prices,price.has_different_original_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,price.total_discount,total_discount
brand_name,,,,,,,,,
Angel & Rocket,5.0,339.75,339.75,0.0,0.0,0.0,0.0,0.0,0.0
Belstaff,2.0,474.90,474.90,0.0,0.0,0.0,0.0,0.0,0.0
Bikkembergs Kids,8.0,644.60,644.60,0.0,0.0,0.0,0.0,0.0,0.0
Bogs,0.0,528.50,528.50,0.0,0.0,0.0,0.0,0.0,0.0
Bomboogie,0.0,1594.55,1594.55,0.0,0.0,0.0,0.0,0.0,0.0
Craft,0.0,71.85,71.85,0.0,0.0,0.0,0.0,0.0,0.0
Enter,0.0,79.90,79.90,0.0,0.0,0.0,0.0,0.0,0.0
Fabrizio,0.0,157.70,157.70,0.0,0.0,0.0,0.0,0.0,0.0
Finger in the nose,1.0,84.95,84.95,0.0,0.0,0.0,0.0,0.0,0.0
